In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np


In [3]:
df = pd.read_csv("../data/CleanedFlights.csv")

dfAirports = pd.read_csv("../data/Airports.csv")
pd.set_option("display.max_columns", None)

dfAirlines = pd.read_csv("../data/airlines.csv")
# creating dictionaries from the IATA_CODE to the Airport, Latitude and Longiture

AirportDict = pd.Series(dfAirports.AIRPORT.values, index = dfAirports.IATA_CODE).to_dict()
AirLineDict = pd.Series(dfAirlines.AIRLINE.values, index = dfAirlines.IATA_CODE).to_dict()
LatitudeDict = pd.Series(dfAirports.LATITUDE.values, index = dfAirports.IATA_CODE).to_dict()
LongitudeDict = pd.Series(dfAirports.LONGITUDE.values, index = dfAirports.IATA_CODE).to_dict()

df.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,FULL_DATE,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,CANCELLED
0,0,2015-01-01,2015,1,1,Thursday,AS,ANC,SEA,00:05,23:54.0,-11.0,205.0,194.0,169.0,1448,430,408.0,-22.0,0
1,1,2015-01-01,2015,1,1,Thursday,AA,LAX,PBI,00:10,00:02.0,-8.0,280.0,279.0,263.0,2330,750,741.0,-9.0,0
2,2,2015-01-01,2015,1,1,Thursday,US,SFO,CLT,00:20,00:18.0,-2.0,286.0,293.0,266.0,2296,806,811.0,5.0,0
3,3,2015-01-01,2015,1,1,Thursday,AA,LAX,MIA,00:20,00:15.0,-5.0,285.0,281.0,258.0,2342,805,756.0,-9.0,0
4,4,2015-01-01,2015,1,1,Thursday,AS,SEA,ANC,00:25,00:24.0,-1.0,235.0,215.0,199.0,1448,320,259.0,-21.0,0


In [4]:
#  df for delay by airport and total flights from this airport

count = pd.DataFrame(df.groupby(["ORIGIN_AIRPORT"]).count()["YEAR"])
df2 = df.groupby(["ORIGIN_AIRPORT"]).mean()
preparedDfForDelayByAirport = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df2.ARRIVAL_DELAY })
preparedDfForDelayByAirport = preparedDfForDelayByAirport.reset_index(drop= False)
preparedDfForDelayByAirport = preparedDfForDelayByAirport.sort_values(by = "AVERAGE_ARRIVAL_DELAY")
preparedDfForDelayByAirport = preparedDfForDelayByAirport[preparedDfForDelayByAirport["TOTAL_FLIGHTS"] > 5000]
preparedDfForDelayByAirport.insert(column = "AIRPORT", loc = 0, value = preparedDfForDelayByAirport.ORIGIN_AIRPORT.map(AirportDict))
preparedDfForDelayByAirport.drop(columns = ["ORIGIN_AIRPORT"])
preparedDfForDelayByAirport.head()

,AIRPORT,ORIGIN_AIRPORT,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
19,NaN,10397,30824,-4.129073
273,NaN,14869,8275,-3.532170
204,NaN,13487,10027,-3.161481
184,NaN,13232,7180,-2.848130
216,NaN,13930,26117,-2.522307


In [5]:
# df for delay by airline
df = pd.read_csv("../data/CleanedFlights.csv")

count = pd.DataFrame(df.groupby(["AIRLINE"]).count()["YEAR"])
df = df.groupby(["AIRLINE"]).mean()
dfDelayByAirline = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByAirline = dfDelayByAirline.reset_index()
dfDelayByAirline["AIRLINE"] = dfDelayByAirline.AIRLINE.map(AirLineDict)

dfDelayByAirline.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,AIRLINE,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
0,American Airlines Inc.,725984,3.451372
1,Alaska Airlines Inc.,172521,-0.976563
2,JetBlue Airways,267048,6.677861
3,Delta Air Lines Inc.,875881,0.186754
4,Atlantic Southeast Airlines,571977,6.585379


In [8]:
# df for delay by weekday 

df = pd.read_csv("../data/CleanedFlights.csv")

count = pd.DataFrame(df.groupby(["DAY_OF_WEEK"]).count()["YEAR"])
df = df.groupby(["DAY_OF_WEEK"]).mean()
dfDelayByWeekday = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByWeekday= dfDelayByWeekday.reset_index()

dfDelayByWeekday.head(8)

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



,DAY_OF_WEEK,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
0,Friday,862209,4.764079
1,Monday,865543,6.002063
2,Saturday,700545,1.852892
3,Sunday,817764,3.958293
4,Thursday,872521,5.668334
5,Tuesday,844600,4.246041
6,Wednessday,855897,3.845468


In [10]:
# df for delay by month

df = pd.read_csv("../data/CleanedFlights.csv")

count = pd.DataFrame(df.groupby(["MONTH"]).count()["YEAR"])
df = df.groupby(["MONTH"]).mean()
dfDelayByMonth = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByMonth= dfDelayByMonth.reset_index()

dfDelayByMonth.head(12)

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



,MONTH,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
0,1,469968,5.813583
1,2,429191,8.320500
2,3,504312,4.920673
3,4,485151,3.163190
4,5,496993,4.485019
5,6,503897,9.601590
6,7,520718,6.431775
7,8,510536,4.607372
8,9,464946,-0.772571
9,10,486165,-0.780390


In [7]:


fig = go.Figure()
scatter = go.Scatter(x= preparedDfForDelayByAirport["TOTAL_FLIGHTS"], y = preparedDfForDelayByAirport["AVERAGE_ARRIVAL_DELAY"], mode = "markers", name = "",
                    customdata = preparedDfForDelayByAirport.AIRPORT,
                    hovertemplate = "%{customdata} <br>Average delay: %{y} min <br>Flights in 2015: %{x}",
                    marker = dict(
                        color = "black",
                        size = 3.5,
                        opacity = 0.5)
                    )

fig.update(layout_xaxis_range = [3000, 350000])
fig.add_trace(scatter)
fig.show()

